In [1]:
# fixed_h5_reader.py - 2025-07-20
"""
Fixed HDF5 data reader that handles both scalar and array datasets correctly.
"""

import h5py
import numpy as np
import pandas as pd
from datetime import datetime
import os

def safe_read_dataset(dataset):
    """Safely read HDF5 dataset, handling both scalar and array data"""
    if dataset.shape == ():  # Scalar dataset
        value = dataset[()]
        if isinstance(value, bytes):
            return value.decode()
        return value
    else:  # Array dataset
        if dataset.dtype.kind in ['S', 'U']:  # String data
            return [item.decode() if isinstance(item, bytes) else str(item) for item in dataset[:]]
        else:
            return dataset[:]

def explore_h5_structure(file_path, max_depth=3):
    """Recursively explore HDF5 file structure"""
    
    def print_item(name, obj, depth=0):
        indent = "  " * depth
        if isinstance(obj, h5py.Group):
            print(f"{indent}📁 {name}/ (Group)")
            if depth < max_depth:
                for key in obj.keys():
                    print_item(key, obj[key], depth + 1)
        elif isinstance(obj, h5py.Dataset):
            shape_str = f"{obj.shape}" if obj.shape != () else "scalar"
            dtype_str = str(obj.dtype)
            print(f"{indent}📄 {name} (Dataset: {shape_str}, {dtype_str})")
    
    with h5py.File(file_path, 'r') as f:
        print(f"📊 HDF5 FILE STRUCTURE: {os.path.basename(file_path)}")
        print("="*60)
        for key in f.keys():
            print_item(key, f[key])

def load_all_data(file_path):
    """Load all data from HDF5 file into organized dictionaries"""
    
    data = {}
    
    with h5py.File(file_path, 'r') as f:
        print(f"🔄 LOADING DATA FROM: {os.path.basename(file_path)}")
        print("="*60)
        
        # Load bus data
        if 'bus' in f:
            bus_data = {}
            for key in f['bus'].keys():
                bus_data[key] = safe_read_dataset(f['bus'][key])
            data['bus'] = bus_data
            print(f"✅ Bus data loaded: {len(bus_data)} attributes, {len(bus_data['name'])} buses")
        
        # Load generator data
        if 'gen' in f:
            gen_data = {}
            for key in f['gen'].keys():
                gen_data[key] = safe_read_dataset(f['gen'][key])
            data['gen'] = gen_data
            print(f"✅ Generator data loaded: {len(gen_data)} attributes, {len(gen_data['name'])} generators")
        
        # Load load data
        if 'load' in f:
            load_data = {}
            for key in f['load'].keys():
                load_data[key] = safe_read_dataset(f['load'][key])
            data['load'] = load_data
            print(f"✅ Load data loaded: {len(load_data)} attributes, {len(load_data['name'])} loads")
        
        # Load power plant data
        if 'power_plants' in f:
            plant_data = {}
            plants_group = f['power_plants']
            
            # Get number of plants
            if 'num_plants' in plants_group:
                num_plants = safe_read_dataset(plants_group['num_plants'])
                plant_data['num_plants'] = num_plants
                print(f"🏭 Found {num_plants} power plants")
                
                # Load individual plant data
                plants = []
                for i in range(int(num_plants)):
                    plant_key = f'plant_{i}'
                    if plant_key in plants_group:
                        plant_info = {}
                        plant_group = plants_group[plant_key]
                        
                        # Load basic plant info
                        for attr in ['name', 'type', 'capacity_MW', 'generators', 'bus_connections']:
                            if attr in plant_group:
                                plant_info[attr] = safe_read_dataset(plant_group[attr])
                        
                        # Load control systems
                        if 'control_systems' in plant_group:
                            controls = []
                            ctrl_group = plant_group['control_systems']
                            
                            # Count control systems
                            ctrl_count = len([k for k in ctrl_group.keys() if k.startswith('control_')])
                            
                            for j in range(ctrl_count):
                                ctrl_key = f'control_{j}'
                                if ctrl_key in ctrl_group:
                                    ctrl_info = {}
                                    ctrl_subgroup = ctrl_group[ctrl_key]
                                    
                                    # Load control basic info
                                    for attr in ['name', 'type']:
                                        if attr in ctrl_subgroup:
                                            ctrl_info[attr] = safe_read_dataset(ctrl_subgroup[attr])
                                    
                                    # Load parameters
                                    if 'parameters' in ctrl_subgroup:
                                        params = {}
                                        param_group = ctrl_subgroup['parameters']
                                        for param_name in param_group.keys():
                                            params[param_name] = safe_read_dataset(param_group[param_name])
                                        ctrl_info['parameters'] = params
                                    
                                    controls.append(ctrl_info)
                            
                            plant_info['control_systems'] = controls
                        
                        plants.append(plant_info)
                
                plant_data['plants'] = plants
            
            data['power_plants'] = plant_data
            print(f"✅ Power plant data loaded: {len(plants)} plants with control systems")
        
        # Load control systems summary
        if 'control_systems_summary' in f:
            ctrl_summary = {}
            summary_group = f['control_systems_summary']
            for key in summary_group.keys():
                ctrl_summary[key] = safe_read_dataset(summary_group[key])
            data['control_summary'] = ctrl_summary
            print(f"✅ Control systems summary loaded")
        
        # Load metadata
        if 'metadata' in f:
            metadata = {}
            meta_group = f['metadata']
            for key in meta_group.keys():
                metadata[key] = safe_read_dataset(meta_group[key])
            data['metadata'] = metadata
            print(f"✅ Metadata loaded: {metadata.get('extraction_date', 'Unknown date')}")
    
    return data

def create_summary_dataframes(data):
    """Create pandas DataFrames for easy analysis"""
    
    dataframes = {}
    
    # Bus DataFrame
    if 'bus' in data:
        bus_df = pd.DataFrame(data['bus'])
        dataframes['buses'] = bus_df
        print(f"📊 Bus DataFrame: {bus_df.shape}")
    
    # Generator DataFrame
    if 'gen' in data:
        gen_df = pd.DataFrame(data['gen'])
        dataframes['generators'] = gen_df
        print(f"📊 Generator DataFrame: {gen_df.shape}")
    
    # Load DataFrame
    if 'load' in data:
        load_df = pd.DataFrame(data['load'])
        dataframes['loads'] = load_df
        print(f"📊 Load DataFrame: {load_df.shape}")
    
    # Power Plant Summary DataFrame
    if 'power_plants' in data and 'plants' in data['power_plants']:
        plants_summary = []
        for i, plant in enumerate(data['power_plants']['plants']):
            summary = {
                'plant_idx': i,
                'name': plant.get('name', 'Unknown'),
                'type': plant.get('type', 'Unknown'),
                'capacity_MW': plant.get('capacity_MW', 0),
                'num_generators': len(plant.get('generators', [])) if isinstance(plant.get('generators', []), list) else 1,
                'num_buses': len(plant.get('bus_connections', [])) if isinstance(plant.get('bus_connections', []), list) else 1,
                'num_controls': len(plant.get('control_systems', []))
            }
            
            # Control system breakdown
            controls = plant.get('control_systems', [])
            summary['avr_count'] = sum(1 for c in controls if 'AVR' in c.get('type', ''))
            summary['gov_count'] = sum(1 for c in controls if 'GOV' in c.get('type', ''))
            summary['pss_count'] = sum(1 for c in controls if 'PSS' in c.get('type', ''))
            
            plants_summary.append(summary)
        
        plants_df = pd.DataFrame(plants_summary)
        dataframes['power_plants'] = plants_df
        print(f"📊 Power Plants DataFrame: {plants_df.shape}")
    
    return dataframes

def analyze_control_systems(data):
    """Detailed analysis of control systems"""
    
    if 'power_plants' not in data or 'plants' not in data['power_plants']:
        print("❌ No power plant data available for control analysis")
        return
    
    print(f"\n🎛️ CONTROL SYSTEMS ANALYSIS")
    print("="*50)
    
    all_controls = []
    
    for plant in data['power_plants']['plants']:
        plant_name = plant.get('name', 'Unknown')
        controls = plant.get('control_systems', [])
        capacity = plant.get('capacity_MW', 0)
        
        print(f"\n🏭 {plant_name}:")
        print(f"   Capacity: {capacity:.1f} MW")
        print(f"   Controls: {len(controls)} systems")
        
        for ctrl in controls:
            ctrl_name = ctrl.get('name', 'Unknown')
            ctrl_type = ctrl.get('type', 'Unknown')
            params = ctrl.get('parameters', {})
            
            print(f"   • {ctrl_name} ({ctrl_type})")
            if params:
                print(f"     Parameters: {len(params)} values")
                # Show first few parameters
                for i, (param, value) in enumerate(params.items()):
                    if i < 3:  # Show first 3 parameters
                        print(f"       {param}: {value}")
                    elif i == 3:
                        print(f"       ... and {len(params)-3} more")
                        break
            else:
                print(f"     No parameters found")
            
            # Add to overall analysis
            all_controls.append({
                'plant': plant_name,
                'control_name': ctrl_name,
                'control_type': ctrl_type,
                'num_parameters': len(params),
                'parameters': params
            })
    
    # Overall statistics
    control_types = {}
    total_params = 0
    
    for ctrl in all_controls:
        ctrl_type = ctrl['control_type']
        if ctrl_type not in control_types:
            control_types[ctrl_type] = 0
        control_types[ctrl_type] += 1
        total_params += ctrl['num_parameters']
    
    print(f"\n📈 CONTROL STATISTICS:")
    print(f"   Total control systems: {len(all_controls)}")
    print(f"   Total parameters: {total_params}")
    for ctrl_type, count in control_types.items():
        print(f"   {ctrl_type}: {count} systems")

def print_data_summary(data, dataframes):
    """Print comprehensive data summary"""
    
    print(f"\n📊 COMPREHENSIVE DATA SUMMARY")
    print("="*60)
    
    # Metadata
    if 'metadata' in data:
        meta = data['metadata']
        print(f"📅 Extraction Date: {meta.get('extraction_date', 'Unknown')}")
        print(f"🏗️ Total Buses: {meta.get('total_buses', 'Unknown')}")
        print(f"🔋 Total Generators: {meta.get('total_generators', 'Unknown')}")
        print(f"🏭 Total Plants: {meta.get('total_plants', 'Unknown')}")
        print(f"⚡ Total Capacity: {meta.get('total_capacity_MW', 'Unknown')} MW")
    
    # Network components
    print(f"\n🔌 NETWORK COMPONENTS:")
    if 'buses' in dataframes:
        print(f"   Buses: {len(dataframes['buses'])}")
        voltage_levels = dataframes['buses']['Un_kV'].value_counts().sort_index(ascending=False)
        print(f"   Voltage levels: {len(voltage_levels)} different levels")
        for voltage, count in voltage_levels.head().items():
            print(f"     {voltage:.1f} kV: {count} buses")
    
    if 'generators' in dataframes:
        gen_df = dataframes['generators']
        print(f"   Generators: {len(gen_df)}")
        print(f"   Total generation: {gen_df['P_MW'].sum():.1f} MW")
        print(f"   Average capacity: {gen_df['P_MW'].mean():.1f} MW")
    
    if 'loads' in dataframes:
        load_df = dataframes['loads']
        print(f"   Loads: {len(load_df)}")
        print(f"   Total load: {load_df['P_MW'].sum():.1f} MW")
    
    # Power plants summary
    if 'power_plants' in dataframes:
        plants_df = dataframes['power_plants']
        print(f"\n🏭 POWER PLANTS SUMMARY:")
        print(f"   Total plants: {len(plants_df)}")
        print(f"   Plants with full control: {(plants_df['num_controls'] >= 3).sum()}")
        print(f"   Average controls per plant: {plants_df['num_controls'].mean():.1f}")
        
        # Plant types
        plant_types = plants_df['type'].value_counts()
        print(f"   Plant types:")
        for plant_type, count in plant_types.items():
            print(f"     {plant_type}: {count} plants")
    
    # Control systems
    if 'control_summary' in data:
        ctrl = data['control_summary']
        print(f"\n🎛️ CONTROL SYSTEMS:")
        print(f"   AVR systems: {ctrl.get('total_avr', 0)}")
        print(f"   Governor systems: {ctrl.get('total_gov', 0)}")
        print(f"   PSS systems: {ctrl.get('total_pss', 0)}")

def show_detailed_plant_info(data):
    """Show detailed information for each power plant"""
    
    if 'power_plants' not in data or 'plants' not in data['power_plants']:
        print("❌ No power plant data available")
        return
    
    print(f"\n🏭 DETAILED POWER PLANT INFORMATION")
    print("="*60)
    
    for i, plant in enumerate(data['power_plants']['plants']):
        name = plant.get('name', f'Plant {i}')
        plant_type = plant.get('type', 'Unknown')
        capacity = plant.get('capacity_MW', 0)
        generators = plant.get('generators', [])
        buses = plant.get('bus_connections', [])
        controls = plant.get('control_systems', [])
        
        print(f"\n🏭 Plant {i}: {name}")
        print(f"   Type: {plant_type}")
        print(f"   Capacity: {capacity:.1f} MW")
        print(f"   Generators: {generators}")
        print(f"   Bus connections: {buses}")
        print(f"   Control systems: {len(controls)}")
        
        # Show control systems
        for j, ctrl in enumerate(controls):
            ctrl_name = ctrl.get('name', f'Control {j}')
            ctrl_type = ctrl.get('type', 'Unknown')
            params = ctrl.get('parameters', {})
            
            print(f"     • {ctrl_name} ({ctrl_type})")
            if params:
                print(f"       Parameters ({len(params)}):")
                for param_name, param_value in params.items():
                    print(f"         {param_name}: {param_value}")
            else:
                print(f"       No parameters")

def main():
    """Main function to explore HDF5 data"""
    
    # File path - adjust as needed
    h5_file = r"enhanced_out\39_Bus_New_England_System_enhanced.h5"
    
    if not os.path.exists(h5_file):
        print(f"❌ File not found: {h5_file}")
        print("Please adjust the file path in the script")
        return None, None
    
    print(f"🚀 ENHANCED HDF5 DATA EXPLORATION (FIXED)")
    print(f"📁 File: {h5_file}")
    print(f"📊 Size: {os.path.getsize(h5_file) / 1024:.1f} KB")
    print(f"⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # 1. Explore file structure
    explore_h5_structure(h5_file)
    print()
    
    # 2. Load all data
    data = load_all_data(h5_file)
    print()
    
    # 3. Create DataFrames
    dataframes = create_summary_dataframes(data)
    print()
    
    # 4. Analyze control systems
    analyze_control_systems(data)
    
    # 5. Print comprehensive summary
    print_data_summary(data, dataframes)
    
    # 6. Show detailed plant information
    show_detailed_plant_info(data)
    
    # 7. Show sample data
    print(f"\n📋 SAMPLE DATA:")
    print("="*40)
    
    if 'buses' in dataframes:
        print(f"\n🔌 Sample Bus Data:")
        print(dataframes['buses'].head()[['name', 'Un_kV', 'area']].to_string(index=False))
    
    if 'generators' in dataframes:
        print(f"\n🔋 Sample Generator Data:")
        print(dataframes['generators'].head()[['name', 'P_MW', 'plant_idx']].to_string(index=False))
    
    if 'power_plants' in dataframes:
        print(f"\n🏭 Power Plant Summary:")
        print(dataframes['power_plants'][['name', 'capacity_MW', 'num_controls']].to_string(index=False))
    
    print(f"\n✅ Data exploration complete!")
    print(f"💡 Use the 'data' and 'dataframes' variables for further analysis")
    
    return data, dataframes

if __name__ == "__main__":
    data, dataframes = main()

🚀 ENHANCED HDF5 DATA EXPLORATION (FIXED)
📁 File: enhanced_out\39_Bus_New_England_System_enhanced.h5
📊 Size: 180.9 KB
⏰ 2025-07-20 18:11:53

📊 HDF5 FILE STRUCTURE: 39_Bus_New_England_System_enhanced.h5
📁 bus/ (Group)
  📄 Un_kV (Dataset: (39,), float64)
  📄 area (Dataset: (39,), float64)
  📄 bustype (Dataset: (39,), int64)
  📄 fn_Hz (Dataset: (39,), float64)
  📄 name (Dataset: (39,), |S20)
  📄 zone (Dataset: (39,), float64)
📁 control_systems_summary/ (Group)
  📄 total_avr (Dataset: scalar, int64)
  📄 total_dsl (Dataset: scalar, int64)
  📄 total_gov (Dataset: scalar, int64)
  📄 total_pss (Dataset: scalar, int64)
📁 gen/ (Group)
  📄 D (Dataset: (10,), float64)
  📄 H_s (Dataset: (10,), float64)
  📄 P_MW (Dataset: (10,), float64)
  📄 Q_MVAR (Dataset: (10,), float64)
  📄 Vset_pu (Dataset: (10,), float64)
  📄 X_d (Dataset: (10,), float64)
  📄 X_q (Dataset: (10,), float64)
  📄 bus_idx (Dataset: (10,), int64)
  📄 model (Dataset: (10,), |S16)
  📄 name (Dataset: (10,), |S24)
  📄 plant_idx (Dataset:

In [2]:
# simple_h5_viewer.py - Quick view of your enhanced HDF5 data
"""
Simple script to quickly view what's in your enhanced HDF5 file.
Run this to see all your extracted data!
"""

import h5py
import numpy as np
import os

def view_h5_data(file_path="enhanced_out/39_Bus_New_England_System_enhanced.h5"):
    """Simple viewer for HDF5 data"""
    
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        print("Make sure the path is correct!")
        return
    
    print(f"🚀 VIEWING HDF5 DATA")
    print(f"📁 File: {file_path}")
    print(f"📊 Size: {os.path.getsize(file_path) / 1024:.1f} KB")
    print("="*60)
    
    with h5py.File(file_path, 'r') as f:
        
        # Show main groups
        print(f"📂 MAIN GROUPS:")
        for group_name in f.keys():
            print(f"   • {group_name}")
        print()
        
        # ═══════════════════════════════════════════════════════════════
        # BUS DATA
        # ═══════════════════════════════════════════════════════════════
        if 'bus' in f:
            bus_grp = f['bus']
            print(f"🔌 BUS DATA ({len(bus_grp['name'])} buses):")
            print(f"   Available attributes: {list(bus_grp.keys())}")
            
            # Show sample bus data
            print(f"   Sample buses:")
            for i in range(min(5, len(bus_grp['name']))):
                name = bus_grp['name'][i].decode() if isinstance(bus_grp['name'][i], bytes) else bus_grp['name'][i]
                voltage = bus_grp['Un_kV'][i]
                print(f"     {i}: {name} ({voltage:.1f} kV)")
            print()
        
        # ═══════════════════════════════════════════════════════════════
        # GENERATOR DATA
        # ═══════════════════════════════════════════════════════════════
        if 'gen' in f:
            gen_grp = f['gen']
            print(f"🔋 GENERATOR DATA ({len(gen_grp['name'])} generators):")
            print(f"   Available attributes: {list(gen_grp.keys())}")
            
            # Show sample generator data
            print(f"   Sample generators:")
            for i in range(min(5, len(gen_grp['name']))):
                name = gen_grp['name'][i].decode() if isinstance(gen_grp['name'][i], bytes) else gen_grp['name'][i]
                power = gen_grp['P_MW'][i]
                plant_idx = gen_grp['plant_idx'][i] if 'plant_idx' in gen_grp else -1
                print(f"     {i}: {name} ({power:.1f} MW, Plant {plant_idx})")
            print()
        
        # ═══════════════════════════════════════════════════════════════
        # LOAD DATA
        # ═══════════════════════════════════════════════════════════════
        if 'load' in f:
            load_grp = f['load']
            print(f"📍 LOAD DATA ({len(load_grp['name'])} loads):")
            print(f"   Available attributes: {list(load_grp.keys())}")
            
            # Show sample load data
            total_load = np.sum(load_grp['P_MW'])
            print(f"   Total load: {total_load:.1f} MW")
            print(f"   Sample loads:")
            for i in range(min(3, len(load_grp['name']))):
                name = load_grp['name'][i].decode() if isinstance(load_grp['name'][i], bytes) else load_grp['name'][i]
                power = load_grp['P_MW'][i]
                print(f"     {i}: {name} ({power:.1f} MW)")
            print()
        
        # ═══════════════════════════════════════════════════════════════
        # POWER PLANT DATA (MOST IMPORTANT!)
        # ═══════════════════════════════════════════════════════════════
        if 'power_plants' in f:
            plants_grp = f['power_plants']
            num_plants = plants_grp['num_plants'][()] if 'num_plants' in plants_grp else 0
            
            print(f"🏭 POWER PLANT DATA ({num_plants} plants):")
            print(f"   Available groups: {list(plants_grp.keys())}")
            
            # Show each power plant
            for i in range(num_plants):
                plant_key = f'plant_{i}'
                if plant_key in plants_grp:
                    plant = plants_grp[plant_key]
                    
                    # Basic plant info
                    name = plant['name'][()].decode() if 'name' in plant else f"Plant {i}"
                    plant_type = plant['type'][()].decode() if 'type' in plant else "Unknown"
                    capacity = plant['capacity_MW'][()] if 'capacity_MW' in plant else 0.0
                    generators = plant['generators'][:] if 'generators' in plant else []
                    buses = plant['bus_connections'][:] if 'bus_connections' in plant else []
                    
                    print(f"\n   🏭 {name} ({plant_type}):")
                    print(f"      Capacity: {capacity:.1f} MW")
                    print(f"      Generators: {len(generators)} units")
                    print(f"      Buses: {buses}")
                    
                    # Control systems
                    if 'control_systems' in plant:
                        ctrl_grp = plant['control_systems']
                        ctrl_count = len([k for k in ctrl_grp.keys() if k.startswith('control_')])
                        print(f"      Control systems: {ctrl_count} systems")
                        
                        # Show each control system
                        for j in range(ctrl_count):
                            ctrl_key = f'control_{j}'
                            if ctrl_key in ctrl_grp:
                                ctrl = ctrl_grp[ctrl_key]
                                ctrl_name = ctrl['name'][()].decode() if 'name' in ctrl else f"Control {j}"
                                ctrl_type = ctrl['type'][()].decode() if 'type' in ctrl else "Unknown"
                                
                                # Parameters
                                param_count = 0
                                if 'parameters' in ctrl:
                                    param_grp = ctrl['parameters']
                                    param_count = len(param_grp.keys())
                                    
                                    print(f"        • {ctrl_name} ({ctrl_type}): {param_count} parameters")
                                    
                                    # Show first few parameters
                                    for k, param_name in enumerate(param_grp.keys()):
                                        if k < 3:  # Show first 3 parameters
                                            param_value = param_grp[param_name][()]
                                            print(f"          {param_name}: {param_value}")
                                        elif k == 3 and len(param_grp.keys()) > 3:
                                            print(f"          ... and {len(param_grp.keys())-3} more")
                                            break
                                else:
                                    print(f"        • {ctrl_name} ({ctrl_type}): No parameters")
            print()
        
        # ═══════════════════════════════════════════════════════════════
        # CONTROL SYSTEMS SUMMARY
        # ═══════════════════════════════════════════════════════════════
        if 'control_systems_summary' in f:
            ctrl_summary = f['control_systems_summary']
            print(f"🎛️ CONTROL SYSTEMS SUMMARY:")
            for key in ctrl_summary.keys():
                value = ctrl_summary[key][()]
                print(f"   {key}: {value}")
            print()
        
        # ═══════════════════════════════════════════════════════════════
        # METADATA
        # ═══════════════════════════════════════════════════════════════
        if 'metadata' in f:
            meta_grp = f['metadata']
            print(f"📋 METADATA:")
            for key in meta_grp.keys():
                value = meta_grp[key][()]
                if isinstance(value, bytes):
                    value = value.decode()
                print(f"   {key}: {value}")
        
    print(f"\n✅ HDF5 data viewing complete!")
    print(f"💡 Your data includes:")
    print(f"   • Complete network topology (buses, generators, loads)")
    print(f"   • Power plant hierarchy with generator assignments")
    print(f"   • Control system details with parameters")
    print(f"   • Ready for ML/GNN applications!")

if __name__ == "__main__":
    # Adjust this path to match your file location
    file_path = r"enhanced_out\39_Bus_New_England_System_enhanced.h5"
    view_h5_data(file_path)

🚀 VIEWING HDF5 DATA
📁 File: enhanced_out\39_Bus_New_England_System_enhanced.h5
📊 Size: 180.9 KB
📂 MAIN GROUPS:
   • bus
   • control_systems_summary
   • gen
   • load
   • metadata
   • power_plants

🔌 BUS DATA (39 buses):
   Available attributes: ['Un_kV', 'area', 'bustype', 'fn_Hz', 'name', 'zone']
   Sample buses:
     0: Bus 01 (nan kV)
     1: Bus 02 (nan kV)
     2: Bus 03 (nan kV)
     3: Bus 04 (nan kV)
     4: Bus 05 (nan kV)

🔋 GENERATOR DATA (10 generators):
   Available attributes: ['D', 'H_s', 'P_MW', 'Q_MVAR', 'Vset_pu', 'X_d', 'X_q', 'bus_idx', 'model', 'name', 'plant_idx']
   Sample generators:
     0: G 01 (1000.0 MW, Plant 9)
     1: G 02 (0.0 MW, Plant 0)
     2: G 03 (650.0 MW, Plant 1)
     3: G 04 (632.0 MW, Plant 2)
     4: G 05 (254.0 MW, Plant 3)

📍 LOAD DATA (19 loads):
   Available attributes: ['P_MW', 'Q_MVAR', 'bus_idx', 'model', 'name']
   Total load: 6097.1 MW
   Sample loads:
     0: Load 03 (322.0 MW)
     1: Load 04 (500.0 MW)
     2: Load 07 (233.8 M